In [1]:
from bs4 import BeautifulSoup
import re

### Extract Book Info

In [16]:
path = '../raw_data_sample/book_info/'
book_names, languages, authors, urls = [], [], [], []
for book_id in range(1, 105):
    html = open('{}/{}.html'.format(path, book_id), 'r')
    soup = BeautifulSoup(html, features='lxml')

    b_id = book_id # book_id
#     try:
#         book_name = soup.find(name='th', text='Title').parent.td.get_text()
#     except AttributeError:
#         book_name = 'None'
#     try:
#         author = soup.find(name='th', text='Author').parent.td.get_text().strip()
#     except AttributeError:
#         author = 'Unknown'
    book_name = soup.find(name='th', text='Title').parent.td.get_text().strip()
    language = soup.find(name='th', text='Language').parent.td.get_text().strip()
    try:
        author = soup.find(name='th', text='Author').parent.td.get_text().strip()
    except AttributeError:
        author = 'Unknown'
    try:
        url = 'https:' + soup.find(text=re.compile('Read this book online')).parent['href']
    except AttributeError:
        url = 'Unknown'

    book_names.append(book_name)
    languages.append(language)
    authors.append(author)
    urls.append(url)

### Extract Figure Info

In [ ]:
path = '../raw_data_sample/book_content/'